# 要約

このノートブックでは、複数のドキュメントに対してLangChainを使用して要約を行う方法について説明します。 `stuff`、`map_reduce`、`refine`の3つの異なるチェーンタイプをカバーしています。これらのチェーンタイプが何であるかの詳細な説明については、[こちら](https://docs.langchain.com/docs/components/chains/index_related_chains)を参照してください。

## データの準備
まず、データを準備します。この例では、1つの長いドキュメントから複数のドキュメントを作成しますが、これらのドキュメントは任意の方法で取得できます（このノートブックのポイントは、ドキュメントを取得した後に何をするかを強調することです）。

In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)

text_splitter = CharacterTextSplitter()

In [ ]:
with open("../../state_of_the_union.txt") as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

In [ ]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:3]]

In [ ]:
from langchain.chains.summarize import load_summarize_chain

## クイックスタート
できるだけ早く始めたい場合は、以下の手順をお勧めします。

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

もし、より多くの制御や理解を望む場合は、以下の情報を参照してください。

## The `stuff` Chain

This sections shows results of using the `stuff` Chain to do summarization.

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff")

In [ ]:
chain.run(docs)

**カスタムプロンプト**

このチェーンでは、独自のプロンプトも使用できます。この例では、日本語で応答します。

In [ ]:
prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY IN JAPANESE:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
chain.run(docs)

## `map_reduce` チェーン

このセクションでは、`map_reduce` チェーンを使用して要約を行った結果を示します。

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce")

In [ ]:
chain.run(docs)

**中間ステップ**

`map_reduce`チェーンの中間ステップを返すこともできます。これは、それらを検査したい場合に行われます。これは、`return_map_steps`変数を使用して行われます。

In [ ]:
chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True)

In [ ]:
chain({"input_documents": docs}, return_only_outputs=True)

**カスタムプロンプト**

このチェーンでは、独自のプロンプトも使用できます。この例では、日本語で返答します。

In [ ]:
prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY IN JAPANESE:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
chain({"input_documents": docs}, return_only_outputs=True)

## `refine` チェーン

このセクションでは、`refine` チェーンを使用して要約を行った結果を示します。

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine", map_prompt=PROMPT, combine_prompt=PROMPT)

chain.run(docs)

**中間ステップ**

`refine`チェーンの中間ステップも返すことができます。これは、それらを検査したい場合に行います。これは、`return_refine_steps`変数を使用して行われます。

In [ ]:
chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="refine", return_intermediate_steps=True)

chain({"input_documents": docs}, return_only_outputs=True)

**カスタムプロンプト**

このチェーンでは、独自のプロンプトも使用できます。この例では、日本語で応答します。

In [ ]:
prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY IN JAPANESE:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Japanese"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(OpenAI(temperature=0), chain_type="refine", return_intermediate_steps=True, question_prompt=PROMPT, refine_prompt=refine_prompt)
chain({"input_documents": docs}, return_only_outputs=True)